In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import os
import logging
import base64
import io
import json
import cv2
import requests
import time  
import sys
import glob
import numpy as np
import ipynb.fs.full.constants as utils

In [ ]:
det_path = utils.detection_path
threshold = utils.threshold
port_number = utils.port_number
video_path = utils.video_path 
every = utils.every
#fps_1 = utils.fps_1
#fps_2 = utils.fps_2
overwrite = utils.overwrite
chunk_size = utils.chunk_size
fps = utils.WRITE_FPS

In [ ]:
def print_progress(iteration, total, prefix='', suffix='', decimals=3, bar_length=100):
    """
    Call in a loop to create standard out progress bar
    :param iteration: current iteration
    :param total: total iterations
    :param prefix: prefix string
    :param suffix: suffix string
    :param decimals: positive number of decimals in percent complete
    :param bar_length: character length of bar
    :return: None
    """

    format_str = "{0:." + str(decimals) + "f}"  # format the % done number string
    percents = format_str.format(100 * (iteration / float(total)))  # calculate the % done
    filled_length = int(round(bar_length * iteration / float(total)))  # calculate the filled bar length
    bar = '#' * filled_length + '-' * (bar_length - filled_length)  # generate the bar string
    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percents, '%', suffix)),  # write out the bar
    sys.stdout.flush()  # flush to stdout

In [ ]:
def extract_frames(video_path, frames_dir,threshold,port_number,overwrite, start, end, every,batch):
    """
    Extract frames from a video using OpenCVs VideoCapture
    :param video_path: path of the video
    :param frames_dir: the directory to save the frames
    :param overwrite: to overwrite frames that already exist?
    :param start: start frame
    :param end: end frame
    :param every: frame spacing
    :return: count of images saved
    """

    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible

    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path

    assert os.path.exists(video_path)  # assert the video file exists

    capture = cv2.VideoCapture(video_path)  # open the video using OpenCV

    if start < 0:  # if start isn't specified lets assume 0
        start = 0
    if end < 0:  # if end isn't specified assume the end of the video
        end = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

    capture.set(1, start)  # set the starting frame of the capture
    frame = start  # keep track of which frame we are up to, starting from start
    #every = frame
    while_safety = 0  # a safety counter to ensure we don't enter an infinite while loop (hopefully we won't need it)
    saved_count = 0  # a count of how many frames we have saved
    count = 0
    previous_detect_count = 0
    #ilename = 1
    #atch_count = 0
    os.makedirs(os.path.join(frames_dir, "{}".format(batch)),exist_ok=True)
    while frame < end:  # lets loop through the frames until the end
        _, image = capture.read()  # read an image from the capture

        if while_safety > 500:  # break the while if our safety maxs out at 500
            break

        # sometimes OpenCV reads None's during a video, in which case we want to just skip
        if image is None:  # if we get a bad return flag or the image we read is None, lets not save
            while_safety += 1  # add 1 to our while safety, since we skip before incrementing our frame variable
            continue  # skip

        if frame % every == 0:  # if this is a frame we want to write out based on the 'every' argument
            every = 5
            #every = frame + 1
            while_safety = 0  # reset the safety count
            #save_path = os.path.join(frames_dir, video_filename, "{}.jpg".format(frame))
            #if not os.path.exists(save_path) or overwrite:  
            detected,img,current_detect_count = container_predict(image,'001',threshold,port_number)
            if detected == True:
                if count<3 and previous_detect_count != current_detect_count:
                    previous_detect_count = 0
                    count+=1
                    save_path = os.path.join(frames_dir, "{}".format(batch), "{}.jpg".format(frame))
                    if not os.path.exists(save_path) or overwrite:
                        cv2.imwrite(save_path, img)
                        #atch_count += 1
                        saved_count += 1  # increment our counter by one
                        #f batch_count>1000:
                         #  filename+=1
                          # batch_count = 0
                           #os.makedirs(os.path.join(frames_dir, "Batch_{}".format(filename)),exist_ok=True)
                elif count == 3:
                    count = 0
                    previous_detect_count = current_detect_count
                  #  every = frame + fps_1
                    
                #else:
                 #   count = 0
                    #temp = frame + 30
                    #every = 15
                    #if every>end:
                    #    break
                  #  every = frame + fps_2       

        frame += 1  # increment our frame count

    capture.release()  # after the while has finished close the capture

    return saved_count  # and return the count of the images we saved

In [ ]:
def video_to_frames(video_path, frames_dir,threshold,port_number,overwrite, every, chunk_size):
    """
    Extracts the frames from a video using multiprocessing
    :param video_path: path to the video
    :param frames_dir: directory to save the frames
    :param overwrite: overwrite frames if they exist?
    :param every: extract every this many frames
    :param chunk_size: how many frames to split into chunks (one chunk per cpu core process)
    :return: path to the directory where the frames were saved, or None if fails
    """

    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible

    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path

    # make directory to save frames, its a sub dir in the frames_dir with the video name
    #os.makedirs(os.path.join(frames_dir, '1'), exist_ok=True)

    capture = cv2.VideoCapture(video_path)  # load the video
    total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))  # get its total frame count
    print(total)
    capture.release()  # release the capture straight away

    if total < 1:  # if video has no frames, might be and opencv error
        print("Video has no frames. Check your OpenCV + ffmpeg installation")
        return None  # return None

    frame_chunks = [[i, i+chunk_size] for i in range(0, total, chunk_size)]  # split the frames into chunk lists
    frame_chunks[-1][-1] = min(frame_chunks[-1][-1], total-1)  # make sure last chunk has correct end frame, also handles case chunk_size < total
    #print(frame_chunks)
    prefix_str = "Extracting frames from {}".format(video_filename)  # a prefix string to be printed in progress bar

    # execute across multiple cpu cores to speed up processing, get the count automatically
    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:

        futures = [executor.submit(extract_frames, video_path, frames_dir, threshold,port_number,overwrite,f[0], f[1],every,batch)
                   for batch,f in enumerate(frame_chunks)]  # submit the processes: extract_frames(...)
        #print(len(frame_chunks))
        for i, f in enumerate(as_completed(futures)):  # as each process completes
            print_progress(i, len(frame_chunks)-1, prefix=prefix_str, suffix='Complete')  # print it's progress

    #return os.path.join(frames_dir, video_filename)  # when done return the directory containing the frames
    #return frames_dir

In [ ]:
def container_predict(image, image_key, threshold,port_number):
    retval, buffer = cv2.imencode('.jpg', image)
    encoded_image = base64.b64encode(buffer).decode('utf-8')
    instances = {
            'instances': [
                    {'image_bytes': {'b64': str(encoded_image)},
                     'key': image_key}
            ]
    }

   
    url = 'http://localhost:{}/v1/models/default:predict'.format(port_number)
    detected = False
    i = 0
    detect_count  = 0
    response = requests.post(url, data=json.dumps(instances))
    response_json=response.json()
    predictions=response_json['predictions'][0]
    #height, width = image.shape[:2]
    for prediction in predictions['detection_scores']:
        if prediction>threshold:
            detected = True
            detect_count+=1
            #box=predictions['detection_boxes'][i]
            #y1=int(box[0]*height)
            #x1=int(box[1]*width)
            #y2=int(box[2]*height)
            #x2=int(box[3]*width)
            #cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
            #cv2.putText(image, str(prediction), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
        i+=1
    return detected,image,detect_count


In [ ]:
if __name__ == '__main__':

    video_to_frames(video_path,det_path,threshold,port_number,overwrite, every,chunk_size)

In [ ]:
import shutil
import os
def batch_alignment(batch_path):
    total = len(os.listdir(batch_path))
    print(total)
    folder = 1
    try:
        os.makedirs(os.path.join(batch_path,"Batch_{}".format(str(folder))))
    except:
        print('Already exist')
    value = 0
    size = 1000
    if os.path.exists(os.path.join(batch_path,"Batch_{}".format(str(folder)))):
        for i in range(0,total):
            dir = os.path.join(batch_path,"{}".format(i))
            file_names = os.listdir(dir)
            length = len(file_names)
            if (value+length)<size:
                value = value + length
                for file_name in file_names:
                    dest = shutil.move(os.path.join(dir,file_name),os.path.join(batch_path,"Batch_{}".format(str(folder))))
                
            else:
                folder+=1
                try:
                    os.makedirs(os.path.join(batch_path,"Batch_{}".format(str(folder))))
                except:
                    print('Already exist')
                for file_name in file_names:
                    dest = shutil.move(os.path.join(dir,file_name),os.path.join(batch_path,"Batch_{}".format(str(folder))))
                value = length
        
        
batch_alignment(det_path)

In [ ]:
import os
def isEmpty(batch_path): 
    if os.path.exists(batch_path):  # Checking if the directory is empty or not 
        total = len(os.listdir(batch_path))
        for i in range(0,1295):
            ls = os.listdir(os.path.join(batch_path,"{}".format(i))) 
            if len(ls) == 0:
                os.rmdir(os.path.join(batch_path,"{}".format(i)))
                print(f'Removing the directory - {i}')

isEmpty(det_path) 
   

In [ ]:
img_array = []
def process(path):
    for i in range(1,12):
        full_path = os.path.join(path,"Batch_{}".format(i))
        print(full_path)
        file_names = os.listdir(full_path)
        for file in file_names:
            image = cv2.imread(os.path.join(full_path,file))
            height, width, layers = image.shape
            size = (width,height)
            img_array.append(image)
        result = cv2.VideoWriter("{}.mp4".format(i),cv2.VideoWriter_fourcc(*'MP4V'), WRITE_FPS, size)
        for i in range(len(img_array)):
            result.write(img_array[i])
        result.release()

        
process(det_path)